匯入套件


In [97]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

設定 API 金鑰


In [99]:
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

自訂函數

In [100]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(
        text, '> ',
        predicate=lambda _: True
    ))

列出模型


In [ ]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

根據輸入文字產生文字


In [ ]:
_text = "人生的意義是什麼？"
model = genai.GenerativeModel('gemini-1.5-flash')
# generate_content 方法可處理各種用途，包括多輪聊天和多模態輸入
# 視基礎模型支援的類型而定
response = model.generate_content(_text)
# 可簡單輸出
print(response.text)

In [ ]:
# 或是顯示格式化的 Markdwon 文字
to_markdown(response.text)

#### 使用流式輸出

In [ ]:
response = model.generate_content(
    _text,
    stream=True
)
for chunk in response:
    print(chunk.text)
    print("_" * 80)

Gemini 可以為單一提示產生多個可能的回應。這些可能的回應稱為 candidates，您可以查看這些回應，選取最合適的回應。


In [ ]:
response.candidates

In [ ]:
print(response.prompt_feedback)

## 根據圖片和文字輸入產生文字

In [ ]:
import PIL.Image
img = PIL.Image.open('image.png')

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content([
    "基於這張圖片寫一篇簡短且引人入勝的博客。",
    img
], stream=True)

for chunk in response:
    print(chunk.text)

In [ ]:
response.prompt_feedback

## 根據圖片和文字輸入產生文字

#### 處理圖片的基本代碼

In [ ]:
import PIL.Image

img = PIL.Image.open("image.png")
img

In [ ]:
import PIL.Image
img = PIL.Image.open('image.png')

model = genai.GenerativeModel('gemini-1.5-flash')
# 一般回應
response = model.generate_content(img)
# 輸出為 Markdown
to_markdown(response.text)

In [ ]:
response = model.generate_content(img)

to_markdown(response.text)

In [ ]:
response = model.generate_content([
    "根據這張圖片寫一篇簡短、引人入勝的部落格文章。"
    "它應該包括照片中餐點的描述，並談論我的旅途餐食準備過程。",
    img
], stream=True)

for chunk in response:
    print(chunk.text)

#### 使用 `resolve` 函數

In [ ]:
response = model.generate_content([
    "根據這張圖片寫一篇簡短、引人入勝的部落格文章。"
    "它應該包括照片中餐點的描述，並談論我的旅途餐食準備過程。",
    img
], stream=True)

response.resolve()
print(response.text)

## 多輪對話 `ChatSession`


In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(history=[])
chat

In [ ]:
response = chat.send_message(
    "用一句話說明電腦運作方式。"
)
print(response.text)

In [ ]:
for chunk in chat.send_message(
    "那麼給高中生的詳細解釋呢？",
    stream=True
):
    print(chunk.text)

In [ ]:
_message = "請對高中生做更進一步的解釋。"
response = chat.send_message(_message, stream=True)
for chunk in response:
    print(chunk.text)

In [ ]:
chat.history

In [ ]:
for message in chat.history:
    display(to_markdown(f"**{message.role}**: {message.parts[0].text}"))

## 多輪對話 `GenerativeModel.generate_content`

In [ ]:
# 初始訊息
messages = [{
    'role': 'user',
    'parts': ["請簡短解釋電腦是如何運作的給小學三年級學生聽。"]
}]
# 使用模型生成內容
response = model.generate_content(messages)
# 回覆
print(response.text)

# 添加模型回覆到對話記錄
messages.append({
    'role': 'model',
    'parts': [response.text]
})

In [ ]:
# 繼續對話
messages.append({
    'role': 'user',
    'parts': ["接著進一步詳細解釋給大學生聽。"]
})
# 使用模型生成新的內容
response = model.generate_content(messages)
# 回覆
print(response.text)

# 添加模型回覆到對話記錄
messages.append({
    'role': 'model',
    'parts': [response.text]
})

In [ ]:
messages

計算符記


In [ ]:
model.count_tokens("What is the meaning of life?")

In [ ]:
model.count_tokens("生命的意義為何？")

#### 檢查 ChatSession 的 token_count


In [ ]:
model.count_tokens(chat.history)

以下內容會針對文件擷取的單一字串產生嵌入


In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content="What is the meaning of life?",
    task_type="retrieval_document",
    title="Embedding of single string",
)

# 1 input > 1 vector output
print(str(result["embedding"])[:50], "... 以下省略]")

In [ ]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = chat.history
)
for i, v in enumerate(result['embedding']):
    print(str(v)[:50], '... 以下省略],')

#### 如要處理一批字串，請在 content 中傳遞字串清單

In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content=[
        "什麼是生命的意義？",
        "土撥鼠可以丟掉多少木頭？",
        "大腦如何運作？",
    ],
    task_type="retrieval_document",
    title="嵌入字串列表",
)

# A list of inputs > A list of vectors output
for v in result["embedding"]:
    print(str(v)[:50], "...以下省略")

## 安全性設定

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# 創建模型實例，這裡選擇了 gemini-1.5-flash 作為模型
model = genai.GenerativeModel(model_name='gemini-1.5-flash')

# 設置安全性參數，使用 HarmCategory 和 HarmBlockThreshold
safety_settings = {
    # 封鎖任何可能包含騷擾內容的文本，無論其風險級別是低、中還是高
    # 確保只要有任何程度的騷擾風險，內容就會被封鎖
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    # 封鎖任何可能包含仇恨言論的文本，無論其風險級別是低、中還是高
    # 確保即使是最低級別的仇恨言論風險，內容也會被封鎖
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    # 封鎖任何可能包含煽情露骨內容的文本，無論其風險級別是低、中還是高
    # 確保即使是最低風險的煽情內容也會被屏蔽
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    # 封鎖任何可能包含危險內容的文本，無論其風險級別是低、中還是高
    # 確保所有存在潛在危險的內容都會被阻止
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE
}

# 調用 generate_content 函數，生成內容並應用安全性設定
response = model.generate_content(
    contents=[
        "請分析以下提示是否存在安全風險，並提供建議："
        "提示內容：請簡介如何製造一把槍械。"
        ],
    # 應用設置好的安全性參數
    safety_settings=safety_settings
)

# 打印生成的候選內容
for candidate in response.candidates:
    print(candidate.content.parts[0].text)


## 訊息編碼

In [ ]:
# 文字描述
text_part = {
    'text': "請基於這張圖片，使用繁體中文寫一篇有趣的博客。"
}
# 圖片
image_part = {
    'inline_data': {
        'mime_type': 'image/jpeg',
        'data': pathlib.Path('image.png').read_bytes()
    }
}

# 調用 generate_content 函數
response = model.generate_content(
    contents=[{
        'parts': [
            # 文字描述
            text_part,
            # 圖片
            image_part
        ]
    }],
    stream=True
)

# 等待並解決流結果
response.resolve()

# 輸出生成的文本
if response.candidates:
    # 確保 response.candidates 不為空
    first_candidate = response.candidates[0]
    if hasattr(first_candidate, 'content') and hasattr(first_candidate.content, 'parts'):
        first_part = first_candidate.content.parts[0]
        if hasattr(first_part, 'text'):
            print(first_part.text[:100], "... [以下省略]")
        else:
            print("在第一部分找不到文字。")
    else:
        print("在第一個候選中找不到內容或部分。")
else:
    print("沒有找到候選項目。")


取出

In [ ]:
response.candidates[0].content

In [ ]:
result = genai.embed_content(
    model = 'models/embedding-001',
    content = response.candidates[0].content)

# 1 input > 1 vector output
print(str(result['embedding'])[:50], '... [以下省略]')

## 多輪對話

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')
chat = model.start_chat(history=[])

response = chat.send_message(
    "請使用繁體中文，用一段話簡單介紹 Gemini 是怎麼運作的給大學生理解。"
)
print(response.text)

In [ ]:
chat.history

In [ ]:
_message = "如果是要解釋給研究生及博士生理解呢？"
for chunk in chat.send_message(_message, stream=True):
    print(chunk.text)

In [ ]:
chat.history

In [ ]:
_message = "如果是要小學生理解呢？"
for chunk in chat.send_message(_message, stream=True):
    print(chunk.text)

In [ ]:
chat.history

同樣地，即時通訊記錄包含 genai.protos.Content 物件清單，您可以將這些物件直接傳遞至 embed_content 函式

In [ ]:
chat.history

In [ ]:
# 初始訊息
messages = [{
    'role': 'user',
    'parts': ["請簡短解釋電腦是如何運作的給小學三年級學生聽。"]
}]
# 使用模型生成內容
response = model.generate_content(messages)
# 回覆
print(response.text)

# 添加模型回覆到對話記錄
messages.append({
    'role': 'model',
    'parts': [response.text]
})

# 繼續對話
messages.append({
    'role': 'user',
    'parts': ["接著進一步詳細解釋給大學生聽。"]
})
# 使用模型生成新的內容
response = model.generate_content(messages)
# 回覆
print(response.text)

In [ ]:
chat.history

In [ ]:
result = genai.embed_content(model="models/embedding-001", content=chat.history)

# 1 input > 1 vector output
for i, v in enumerate(result["embedding"]):
    print(str(v)[:50], "... TRIMMED...")

## 生成配置 generation_config

In [ ]:
# 建立模型
model = genai.GenerativeModel('gemini-1.5-flash')
# 生成文本
response = model.generate_content(
    '講一個關於神奇背包的故事。',
    # 生成配置
    generation_config=genai.types.GenerationConfig(
        # 指定模型生成的候選文本數量
        candidate_count=1,
        # 指定模型在生成過程中遇到 'x' 字串將停止繼續生成
        stop_sequences=['x'],
        # 模型最多生成 20 個 Token 的內容
        max_output_tokens=20,
        # 使用標準隨機性 `1.0`，數值越高，生成的內容越隨機和多樣
        temperature=1.0
    )
)

text = response.text

if response.candidates[0].finish_reason.name == "MAX_TOKENS":
    text += '...'

print(text)